In [1]:
!date

Tue Mar 27 10:54:16 CST 2018


In [2]:
!ls ../source/gdppc_cppp/

README.md                      ~$gdppc_cppp-by-gapminder.xlsx
gdppc_cppp-by-gapminder.xlsx


In [3]:
!ls ../../

README.md
datapackage.json
datapackage.json.bak
ddf--concepts.csv
ddf--datapoints--gdp_per_capita_cppp--by--country--year.csv
ddf--datapoints--gdp_per_capita_cppp--by--global--year.csv
ddf--datapoints--gdp_per_capita_cppp--by--world_4region--year.csv
ddf--entities--geo--country.csv
ddf--entities--geo--global.csv
ddf--entities--geo--world_4region.csv
etl


In [4]:
import pandas as pd

In [8]:
# code is not there yet. will add it manually

code = 'gdp_per_capita_cppp'

In [12]:
domain = [ {'concept': 'geo', 'name': 'Geo', 'domain': None, 'concept_type': 'entity_domain'}
         , {'concept': 'country', 'name': 'Country', 'domain': 'geo', 'concept_type': 'entity_set'}
         , {'concept': 'world_4region', 'name': 'four regions', 'domain': 'geo', 'concept_type': 'entity_set'}
         , {'concept': 'global', 'name': 'global', 'domain': 'geo', 'concept_type': 'entity_set'}
         , {'concept': 'domain', 'name': 'Domain', 'concept_type': 'string'}
         ]

domain = pd.DataFrame.from_records(domain)

In [13]:
domain = domain.set_index('concept')

In [14]:
domain

,concept_type,domain,name
concept,,,
geo,entity_domain,None,Geo
country,entity_set,geo,Country
world_4region,entity_set,geo,four regions
global,entity_set,geo,global
domain,string,NaN,Domain


In [15]:
time = pd.DataFrame.from_records([{'concept': 'year', 'concept_type': 'time', 'name': 'Year'}]).set_index('concept')

In [16]:
time

,concept_type,name
concept,,
year,time,Year


In [17]:
measure = pd.DataFrame.from_records(
    [{'concept': code, 'concept_type': 'measure', 'name': 'GDP per Captia'}]).set_index('concept')

In [18]:
strings = pd.DataFrame.from_records(
    [{'concept': 'name', 'concept_type': 'string', 'name': 'Name'}]).set_index('concept')

In [19]:
cdf = pd.concat([measure, strings, domain, time])

In [20]:
cdf = cdf.reset_index()

In [158]:
# cdf['name'] = cdf['name'].fillna(cdf['concept'].map(lambda x: x.replace('_', ' ').title()))

In [21]:
cdf

,concept,concept_type,domain,name
0,gdp_per_capita_cppp,measure,NaN,GDP per Captia
1,name,string,NaN,Name
2,geo,entity_domain,None,Geo
3,country,entity_set,geo,Country
4,world_4region,entity_set,geo,four regions
5,global,entity_set,geo,global
6,domain,string,NaN,Domain
7,year,time,NaN,Year


In [22]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

In [23]:
# datapoints

In [24]:
country_data = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', sheet_name='countries_and_territories')

In [25]:
country_data.head()

,geo.name,indicator.name,geo,indicator,1800,1801,1802,1803,1804,1805,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Abkhazia,"GDP per capita, PPP (constant 2011 internation...",abkh,gdppc_cppp,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,"GDP per capita, PPP (constant 2011 internation...",afg,gdppc_cppp,603.0,603.0,603.0,603.0,603.0,603.0,...,2416.0,2469.0,2524.0,2580.0,2637.0,2695.0,2755.0,2816.0,2878.0,2942.0
2,Akrotiri and Dhekelia,"GDP per capita, PPP (constant 2011 internation...",akr_a_dhe,gdppc_cppp,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,"GDP per capita, PPP (constant 2011 internation...",alb,gdppc_cppp,667.0,667.0,667.0,667.0,667.0,668.0,...,18471.0,18881.0,19299.0,19727.0,20164.0,20611.0,21068.0,21535.0,22012.0,22500.0
4,Algeria,"GDP per capita, PPP (constant 2011 internation...",dza,gdppc_cppp,715.0,716.0,717.0,718.0,719.0,720.0,...,15603.0,15948.0,16301.0,16662.0,17031.0,17408.0,17794.0,18188.0,18591.0,19003.0


In [26]:
# country entity
countries = country_data[['geo', 'geo.name']].copy()

In [27]:
countries.columns = ['country', 'name']

In [28]:
countries.to_csv('../../ddf--entities--geo--country.csv', index=False)

In [29]:
country_df = country_data.set_index('geo').drop(['geo.name', 'indicator', 'indicator.name'], axis=1)

In [30]:
country_df = country_df.stack().reset_index()

In [31]:
country_df.columns = ['country', 'year', code]

In [32]:
country_df.dropna(how='any').to_csv('../../ddf--datapoints--{}--by--country--year.csv'.format(code), index=False)

In [33]:
# 4 region
region_data = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', 
                            sheet_name='four_regions')

In [34]:
region = region_data[['geo', 'geo.name']]
region.columns = ['world_4region', 'name']

In [35]:
region.to_csv('../../ddf--entities--geo--world_4region.csv', index=False)

In [36]:
region_df = region_data.set_index('geo').drop(['geo.name', 'indicator', 'indicator.name'], axis=1)
region_df = region_df.stack().reset_index()
region_df.columns = ['world_4region', 'year', code]

In [37]:
region_df.dropna(how='any').to_csv('../../ddf--datapoints--{}--by--world_4region--year.csv'.format(code), index=False)

In [38]:
# world

world_data = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', 
                            sheet_name='world_total')

In [39]:
world = world_data[['geo', 'geo.name']]
world.columns = ['global', 'name']

world.to_csv('../../ddf--entities--geo--global.csv', index=False)

In [40]:
world_df = world_data.set_index('geo').drop(['geo.name', 'indicator', 'indicator.name'], axis=1)
world_df = world_df.stack().reset_index()
world_df.columns = ['global', 'year', code]

In [41]:
world_df.dropna(how='any').to_csv('../../ddf--datapoints--{}--by--global--year.csv'.format(code), index=False)

In [42]:
!ddf create_datapackage --overwrite ../../

backing up previous datapackage.json...
2018-03-27 11:27:09 WARNING skipping file .DS_Store
2018-03-27 11:27:09 WARNING skipping file datapackage.json
2018-03-27 11:27:09 WARNING skipping file .gitmodules
2018-03-27 11:27:09 WARNING skipping file README.md
2018-03-27 11:27:09 WARNING skipping file .gitignore
2018-03-27 11:27:09 WARNING skipping file datapackage.json.bak
2018-03-27 11:27:09 INFO generating ddf schema, may take some time...
2018-03-27 11:27:09 INFO loading dataset from disk: ddf--gapminder--gdp_per_capita_cppp
100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 40.88it/s]
Done.


In [43]:
!validate-ddf ../../

7root checking   0% /                                                           87root checking 100%[==========================================================>]87dataset loading   0% /                                                         87datapoints validation   0% /                                                   87datapoints validation  67%[=================================>                 ]87datapoints validation 100%[==================================================>]87datapoints validation 100%[==================================================>]8
Validation was finished successfully.
Current version 1.12.0 is too old. New version is 1.14.2. Update please!
